# Installing Packages and Dependencies

In [1]:
!pip install tensorflow-text==2.4.2
!pip install tf-models-official==2.4
!pip install tensorflow-gpu==2.4.2
!pip install delayed
!pip install alt-profanity-check
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
import joblib
from profanity_check import predict, predict_prob
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

     |████████████████████████████████| 3.4 MB 920 kB/s            
     |████████████████████████████████| 394.5 MB 34 kB/s              
     |████████████████████████████████| 2.9 MB 52.2 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 462 kB 47.7 MB/s            
     |████████████████████████████████| 3.8 MB 49.5 MB/s            
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=77049 sha256=670bafbabe05444c32cd4c945296f5e2bea82cef9203b7c67effa0f3698da0cf
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Succe

2022-03-21 06:32:28.794504: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to un

In [2]:
print(tf.__version__)

2.4.2


In [3]:
!nvidia-smi 

Mon Mar 21 06:32:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
tf.test.is_gpu_available()

2022-03-21 06:32:37.059543: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


True

2022-03-21 06:32:37.060213: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-21 06:32:37.061875: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-21 06:32:37.105141: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 06:32:37.105785: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-03-21 06:32:37.105879: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-21 06:32:37.105971: I tensorflow/stream_executor/platform/def

# Reading Training Data

In [5]:
df = pd.read_csv("../input/covid-misinfo/covid19_tweets_unlabelled_data.csv")
df.head()

,id,tweet
0,1,If I smelled the scent of hand sanitizers toda...
1,2,Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2,3,@diane3443 @wdunlap @realDonaldTrump Trump nev...
3,4,@brookbanktv The one gift #COVID19 has give me...
4,5,25 July : Media Bulletin on Novel #CoronaVirus...


In [6]:
df.shape

(178682, 2)

# Data Insight

In [7]:
# df.groupby('label').describe()

In [8]:
# plt.figure(figsize=(8,6))
# sns.set_style(style='darkgrid')
# sns.countplot(df['label'])
# plt.title("Covid Fake and Real Tweets")
# plt.show()

In [9]:
# plt.figure(figsize=(6,8))
# sns.set_style("darkgrid")
# df["label"].value_counts().plot.pie(autopct='%0.2f%%')
# plt.title("Percent contribution")
# plt.show()

# Feature Engineering

In [10]:
def get_prafanity_scores(text):
    embeds = []
    
    for line in tqdm(text):
        
        embeds.append(predict_prob([line])[0])
    
    assert len(text) == len(embeds)    
    return embeds

In [11]:
#textural features
def get_text_ft(text):
    tweet_len =  len(text)
    tweet_num_of_upper_char = sum([len([char for char in word if char.isupper()]) for word in text])
    tweet_num_of_qmarks = sum([len([char for char in word if char == '?']) for word in text]) 
    tweet_num_of_exclmmarks = sum([len([char for char in word if char == '!']) for word in text])
    return tweet_len, tweet_num_of_upper_char, tweet_num_of_qmarks, tweet_num_of_exclmmarks

In [12]:
df['profanity_scores'] = get_prafanity_scores(df['tweet'])
df[['length','num_of_upper_char','num_of_qmarks','num_of_exclmmarks']] = df['tweet'].apply(lambda x: get_text_ft(x)).to_list()

100%|██████████| 178682/178682 [05:29<00:00, 541.69it/s]


In [13]:
df

,id,tweet,profanity_scores,length,num_of_upper_char,num_of_qmarks,num_of_exclmmarks
0,1,If I smelled the scent of hand sanitizers toda...,0.015757,142,7,0,0
1,2,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,0.012887,142,12,0,0
2,3,@diane3443 @wdunlap @realDonaldTrump Trump nev...,0.005394,142,13,0,0
3,4,@brookbanktv The one gift #COVID19 has give me...,0.033829,141,12,0,0
4,5,25 July : Media Bulletin on Novel #CoronaVirus...,0.081944,140,24,0,0
...,...,...,...,...,...,...,...
178677,178678,Thanks @IamOhmai for nominating me for the @WH...,0.004474,130,15,0,0
178678,178679,2020! The year of insanity! Lol! #COVID19 http...,0.007427,65,9,0,3
178679,178680,@CTVNews A powerful painting by Juan Lucena. I...,0.028688,142,15,0,0
178680,178681,"More than 1,200 students test positive for #CO...",0.056671,106,13,0,0


In [14]:
df.nlargest(5, 'profanity_scores')

,id,tweet,profanity_scores,length,num_of_upper_char,num_of_qmarks,num_of_exclmmarks
85539,85540,"Fucking stupid ass idiots, #COVID19 just wear ...",1.0,142,17,0,3
22729,22730,Okay #snapchat is a piece of fucking shit. It'...,1.0,142,10,0,0
89557,89558,"Fuck you, you putrid cunt #Xinjiang #eatshit #...",1.0,123,18,0,0
12294,12295,@SenRickScott @marcorubio @GovRonDeSantis \n\n...,1.0,96,26,0,0
30278,30279,"Fuck you, Gomer! #COVID19 #TrumpVirus https://...",1.0,61,15,0,1


In [15]:
df.nlargest(5, 'num_of_qmarks')

,id,tweet,profanity_scores,length,num_of_upper_char,num_of_qmarks,num_of_exclmmarks
22720,22721,Got kids?\n\nHow many?\n\nParents?\n\nGot savi...,0.012973,120,14,10,0
144114,144115,@SpeakerPelosi Wat about ur gd assault on amer...,0.236682,85,11,9,0
65900,65901,Di ko talaga gets bakit gabi ang press con sa ...,0.020545,141,13,8,0
22547,22548,Paying tuition?\n\nGetting a paycheck?\n\nUnem...,0.044220,142,13,7,0
100683,100684,@nypost When will Jack McCoy indict Cuomo for ...,0.040171,91,13,7,0


# Normalizing

In [16]:
df['length'] = MinMaxScaler().fit_transform(np.array(df['length']).reshape(-1,1))
df['num_of_upper_char'] = MinMaxScaler().fit_transform(np.array(df['num_of_upper_char']).reshape(-1,1))
df['num_of_qmarks'] = MinMaxScaler().fit_transform(np.array(df['num_of_qmarks']).reshape(-1,1))
df['num_of_exclmmarks'] = MinMaxScaler().fit_transform(np.array(df['num_of_exclmmarks']).reshape(-1,1))

In [17]:
df

,id,tweet,profanity_scores,length,num_of_upper_char,num_of_qmarks,num_of_exclmmarks
0,1,If I smelled the scent of hand sanitizers toda...,0.015757,0.320896,0.066038,0.0,0.000000
1,2,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,0.012887,0.320896,0.113208,0.0,0.000000
2,3,@diane3443 @wdunlap @realDonaldTrump Trump nev...,0.005394,0.320896,0.122642,0.0,0.000000
3,4,@brookbanktv The one gift #COVID19 has give me...,0.033829,0.318408,0.113208,0.0,0.000000
4,5,25 July : Media Bulletin on Novel #CoronaVirus...,0.081944,0.315920,0.226415,0.0,0.000000
...,...,...,...,...,...,...,...
178677,178678,Thanks @IamOhmai for nominating me for the @WH...,0.004474,0.291045,0.141509,0.0,0.000000
178678,178679,2020! The year of insanity! Lol! #COVID19 http...,0.007427,0.129353,0.084906,0.0,0.136364
178679,178680,@CTVNews A powerful painting by Juan Lucena. I...,0.028688,0.320896,0.141509,0.0,0.000000
178680,178681,"More than 1,200 students test positive for #CO...",0.056671,0.231343,0.122642,0.0,0.000000


# Data Preprocessing

In [18]:
#Data Cleaning
ps = PorterStemmer()
nltk.download("stopwords")
en_stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
def cleaner(text):
    try:
        url_re = re.compile(r'(http|ftp|https)://([\w+?\.\w+])+([a-zA-Z0-9\~\!\@\#\$\%\^\&\*\(\)_\-\=\+\\\/\?\.\:\;\'\,]*)?')
        cleaned = url_re.sub("",text)
        
        hashtag_re = re.compile(r'(#\w+)')
        cleaned = hashtag_re.sub("",cleaned)
        
        screen_name_re = re.compile(r'(@\w+)')
        cleaned = screen_name_re.sub("", cleaned)
        
        emoji_re = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U0001F1F2-\U0001F1F4"  # Macau flag
                u"\U0001F1E6-\U0001F1FF"  # flags
                u"\U0001F600-\U0001F64F"
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u"\U0001F1F2"
                u"\U0001F1F4"
                u"\U0001F620"
                u"\u200d"
                u"\u2640-\u2642"
                "]+", flags=re.UNICODE)
        cleaned = emoji_re.sub("",cleaned)
        
        cleaned = cleaned.replace("."," ").replace("//"," ")
        cleaned = re.sub(r'[^a-zA-Z]', " ", cleaned) #Replacing all non-alphabets with " "
        cleaned = re.sub(r'\w+\d+', " ", cleaned) #Removes alphanumeric words

        cleaned = ps.stem(cleaned)
        
        cleaned = " ".join([word for word in cleaned.split() if len(word)>2]) #Removing words with len<2
        cleaned = ' '.join([word for word in cleaned.split() if word not in en_stopwords])

        cleaned = cleaned.strip() #Removing whitespaces
        cleaned = cleaned.lower() #Converting to lower case

    except:
        cleaned = ""
    return cleaned

In [20]:
df['tweet'] = df['tweet'].apply(lambda text: cleaner(text))
df.head()

,id,tweet,profanity_scores,length,num_of_upper_char,num_of_qmarks,num_of_exclmmarks
0,1,smelled scent hand sanitizers today someone pa...,0.015757,0.320896,0.066038,0.0,0.0
1,2,hey made sense players pay respects,0.012887,0.320896,0.113208,0.0,0.0
2,3,trump never claimed hoax claim effort,0.005394,0.320896,0.122642,0.0,0.0
3,4,one gift give appreciation simple things alway...,0.033829,0.318408,0.113208,0.0,0.0
4,5,july media bulletin novel,0.081944,0.315920,0.226415,0.0,0.0


# Generating Word Clouds

In [21]:
# #Function to get word frequencies in the data
# def get_word_freqs(df, col):
#     text = ' '.join(df[col])
#     text = text.split()
#     freq = pd.Series(text).value_counts()
#     return freq

In [22]:
# freq_words_fake = get_word_freqs(df[df['label']=='fake'],'tweet')
# freq_words_fake = " ".join(freq_words_fake.index)
# word_cloud=WordCloud(max_font_size=60,background_color='white').generate(freq_words_fake)
# plt.figure(figsize=(8,6))
# plt.imshow(word_cloud)
# plt.axis('off')
# plt.show()

In [23]:
# freq_words_real = get_word_freqs(df[df['label']=='real'],'tweet')
# freq_words_real = " ".join(freq_words_real.index)
# word_cloud=WordCloud(max_font_size=60,background_color='white').generate(freq_words_real)
# plt.figure(figsize=(8,6))
# plt.imshow(word_cloud)
# plt.axis('off')
# plt.show()

In [24]:
# #splitting the labels
# df_real = df[df['label']=='real']
# df_real.shape

In [25]:
# df_fake = df[df['label']=='fake']
# df_fake.shape

# Downsampling Data

In [26]:
#unbalanced data so we downsample it
# df_real_downsampled = df_real.sample(df_fake.shape[0])
# df_real_downsampled.shape

In [27]:
# df_balanced = pd.concat([df_fake, df_real_downsampled])
# df_balanced['label'].value_counts()

In [28]:
# df_balanced['fake'] = df_balanced['label'].apply(lambda x: 1 if x == 'fake' else 0)
# df_balanced.sample(5)

In [29]:
df_features = df[["profanity_scores", "length","num_of_upper_char", "num_of_qmarks", "num_of_exclmmarks"]]
df_features

,profanity_scores,length,num_of_upper_char,num_of_qmarks,num_of_exclmmarks
0,0.015757,0.320896,0.066038,0.0,0.000000
1,0.012887,0.320896,0.113208,0.0,0.000000
2,0.005394,0.320896,0.122642,0.0,0.000000
3,0.033829,0.318408,0.113208,0.0,0.000000
4,0.081944,0.315920,0.226415,0.0,0.000000
...,...,...,...,...,...
178677,0.004474,0.291045,0.141509,0.0,0.000000
178678,0.007427,0.129353,0.084906,0.0,0.136364
178679,0.028688,0.320896,0.141509,0.0,0.000000
178680,0.056671,0.231343,0.122642,0.0,0.000000


In [30]:
# df_test = pd.read_csv("../input/covid-misinfo/Constraint_English_Test.csv")
# df_test['tweet'] = df_test['tweet'].apply(lambda text: cleaner(text))
# df_test['fake'] = df_test['label'].apply(lambda x: 1 if x == 'fake' else 0)
# X_test = df_test[['tweet']] 
# y_test = df_test[['fake']]

In [31]:
# #Feature Extraction
# df_test['profanity_scores'] = get_prafanity_scores(df_test['tweet'])
# df_test[['length','num_of_upper_char','num_of_qmarks','num_of_exclmmarks']] = df_test['tweet'].apply(lambda x: get_text_ft(x)).to_list()

# #Normalizing
# df_test['length'] = MinMaxScaler().fit_transform(np.array(df_test['length']).reshape(-1,1))
# df_test['num_of_upper_char'] = MinMaxScaler().fit_transform(np.array(df_test['num_of_upper_char']).reshape(-1,1))
# df_test['num_of_qmarks'] = MinMaxScaler().fit_transform(np.array(df_test['num_of_qmarks']).reshape(-1,1))
# df_test['num_of_exclmmarks'] = MinMaxScaler().fit_transform(np.array(df_test['num_of_exclmmarks']).reshape(-1,1))

# df_test_features = df_test[["profanity_scores", "length","num_of_upper_char", "num_of_qmarks", "num_of_exclmmarks"]]

# Model Construction

In [32]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
    
    return np.array(all_tokens)

In [33]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    
    feature_input = tf.keras.layers.Input((5,))
    concat= tf.keras.layers.Concatenate()([cls_token, feature_input])
    
    dropout1 = tf.keras.layers.Dropout(0.1, name='dropout1')(concat)
    fc1 = tf.keras.layers.Dense(512, activation='relu', name='fullyConnected1')(dropout1)
    dropout2 = tf.keras.layers.Dropout(0.1, name='dropout2')(fc1)
    fc2 = tf.keras.layers.Dense(256, activation='relu', name='fullyConnected2')(dropout2)
    out = Dense(1, activation='sigmoid')(fc2)
    
    model = Model(inputs=[input_word_ids,feature_input], outputs=out)
    
    return model


In [34]:
%%time
transformer_layer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

2022-03-21 06:40:18.509908: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-21 06:40:18.510540: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 06:40:18.511248: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-03-21 06:40:18.511359: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-21 06:40:18.511423: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-03-21 06:40:18.511446: I tensorflow/stream_executor/platform/

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

CPU times: user 8.07 s, sys: 1.45 s, total: 9.52 s
Wall time: 15.6 s


In [35]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model = build_model(transformer_layer, max_len=160)
model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=METRICS)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 66362880    input_word_ids[0][0]             
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           tf_distil_bert_model[0][0]       
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 5)]          0                                            
______________________________________________________________________________________________

In [36]:
model.load_weights('../input/covid-misinfo/Final_DistilBERT_Text_Ft_10K_Weights.h5')

2022-03-21 06:40:37.373968: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 93763584 exceeds 10% of free system memory.


In [37]:
test_input = bert_encode(df.tweet.values, tokenizer, max_len=160)

In [38]:
# train_input = bert_encode(df_balanced.tweet.values, tokenizer, max_len=160)
# test_input = bert_encode(df_test.tweet.values, tokenizer, max_len=160)
# train_labels = df_balanced.fake.values

In [39]:
# callbacks = [
#     tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode="min"),
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
#                               mode='min',
#                               verbose=1,
#                               patience=3,
#                               min_delta=0.0001,
#                               factor=0.2
#                              )
# ]

In [40]:
# train_history = model.fit(
#     x=[train_input, df_features], y=train_labels,
#     validation_split=0.25,
#     epochs=100,
#     batch_size=32,
#     callbacks = callbacks, verbose=1
# )

In [41]:
pred_prob = model.predict(x=[test_input, df_features])
pred_prob

2022-03-21 06:42:03.246733: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 228712960 exceeds 10% of free system memory.
2022-03-21 06:42:03.467605: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-21 06:42:03.468729: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000179999 Hz


array([[0.99976665],
       [0.9978326 ],
       [0.99802136],
       ...,
       [0.9997713 ],
       [0.999841  ],
       [0.97678965]], dtype=float32)

In [42]:
pred_class  = np.where(pred_prob >0.5, "fake", "real")
predictions = pd.DataFrame(pred_class, columns=['prediction'])

In [43]:
df_org = pd.read_csv("../input/covid-misinfo/covid19_tweets_unlabelled_data.csv")
df_new = pd.concat([df_org, predictions], axis =1)

In [44]:
df_new.to_csv('covid19_tweets_unlabelled_data_predicted.csv')

In [45]:
# test_loss, test_accuracy, test_precision, test_recall = model.evaluate(x=[test_input, df_test_features], y=df_test[['fake']])

# print("Test Loss: ", test_loss)
# print("Test Precision: ", test_precision)
# print("Test Recall: ", test_recall)
# print("Test Accuracy: ", test_accuracy)